In [24]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [4]:
ratio_index_saaty = np.array([0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.46, 1.49]) 

In [18]:
def process(filename, factor):
    p_crit = pd.read_excel(filename, sheet_name=factor).iloc[:, 1:].values
    p_alts = []
    for i in range(1, p_crit.shape[0] + 1):
        p_alts.append(pd.read_excel("ahp1.xlsx", sheet_name=f"F{factor[1]}.{i}").iloc[:, 1:].values)

    p_alts = np.array(p_alts)

    n = p_crit.shape[0]
    p_crit_total = np.sum(p_crit, axis=0)
    p_crit_norm = np.mean(p_crit / p_crit_total, axis=1)

    consistency_measure = np.sum(p_crit_norm * p_crit, axis=1) / p_crit_norm
    print(consistency_measure)
    lambda_max = np.sum(consistency_measure) / n
    consistency_index = (lambda_max - n) / (n - 1)
    ratio_index = consistency_index / ratio_index_saaty[n-1]

    print("Consistency index:", consistency_index)
    print("Ratio index:", ratio_index)

    ranks_norm = []
    for p_alt in p_alts[:]:
        ranks_norm.append(np.mean(p_alt / np.sum(p_alt, axis=0), axis=1).reshape(-1, 1))

    nilai_alternatif = np.sum(p_crit_norm * np.hstack(ranks_norm), axis=1)
    ranking = list(zip(range(1, n+1), nilai_alternatif))
    ranking.sort(key=lambda x: x[1], reverse=True)

    return ranking

In [9]:
ranks = []
for factor in ["F1", "F2", "F3", "F4", "F5"]:
    ranks.extend([ranking[1] for ranking in process("ahp1.xlsx", factor)])

[6.78673081 7.87726286 8.60868192 6.3782875 ]
Consistency index: 1.1375802571471645
Ratio index: 1.2639780634968494
[2. 2.]
Consistency index: 0.0
Ratio index: nan
[2. 2.]
Consistency index: 0.0
Ratio index: nan
[2. 2.]
Consistency index: 0.0
Ratio index: nan


/tmp/ipykernel_23305/2573642831.py:17: RuntimeWarning: invalid value encountered in double_scalars
  ratio_index = consistency_index / ratio_index_saaty[n-1]
/tmp/ipykernel_23305/2573642831.py:17: RuntimeWarning: invalid value encountered in double_scalars
  ratio_index = consistency_index / ratio_index_saaty[n-1]
/tmp/ipykernel_23305/2573642831.py:17: RuntimeWarning: invalid value encountered in double_scalars
  ratio_index = consistency_index / ratio_index_saaty[n-1]


[11.27919332  9.91989381 10.14955644  9.60668874  9.15341835 10.56867221
 10.00419414]
Consistency index: 0.5162289765544941
Ratio index: 0.3910825579958288


## TRIK KEDUA HEHE

In [25]:
def process2(filename, factor):
    p_crit = pd.read_excel(filename, sheet_name=factor).iloc[:, 1:].values
    p_alts = None
    for i in range(1, p_crit.shape[0] + 1):
        if p_alts is None:
            p_alts = pd.read_excel(filename, sheet_name=f"F{factor[1]}.{i}").iloc[:, 1:].values
        else:
            p_alts = p_alts * pd.read_excel(filename, sheet_name=f"F{factor[1]}.{i}").iloc[:, 1:].values

    scaler = MinMaxScaler()
    results = scaler.fit_transform(p_alts)

    return results.mean(axis=1)

In [32]:
ranks = []
for factor in ["F1", "F2", "F3", "F4", "F5"]:
    ranks.append(process2("ahp1.xlsx", factor))

In [49]:
arr = np.array(ranks)
final_geom = arr.prod(axis=0) ** (1/arr.shape[0])
final_geom = final_geom / final_geom.sum()
final_geom

array([0.41196572, 0.21185593, 0.09976493, 0.        , 0.13974151,
       0.13667191])